In [25]:
from pymongo import MongoClient
import requests as req
import pandas as pd
import warnings
import time
import json
import sys

In [2]:
sys.path.append('../src')

from API_support import *

# Enriquecimiento de datos con Deezer API

In [3]:
df = pd.read_csv('../../sonic_harbor.inven.csv')

In [5]:
# df['song_rank'] = df['album'].apply(get_rank)

In [91]:
df.head()

,_id,author,album,label,item_condition,sleeve_condition,price,ship_price,final_price,song_rank
0,6544ebd00d20ed70b40d2a9a,Kid Rock,The History Of Rock,"Lava, Lava, Atlantic, Top Dog Records",Near Mint (NM or M-),Near Mint (NM or M-),5.99,3.6,9.59,"{'Honeycomb': 13168, 'Honeycomb (1957)': 425, ..."
1,6544ebd00d20ed70b40d2a9b,Kid Rock (2),Ic,"Poplandia, Young Blood",Near Mint (NM or M-),Very Good Plus (VG+),34.99,3.6,38.59,"{'I Only Have Eyes for You': 100000, 'Chrome H..."
2,6544ebd00d20ed70b40d2a9c,Olga Ramos,Antologia Del Cuplé,Movieplay,Very Good Plus (VG+),Very Good Plus (VG+),2.99,3.6,6.59,"{'Colón 34': 19610, 'Ven y ven (Versión picare..."
3,6544ebd00d20ed70b40d2a9d,Graham Parker,Protection,Vertigo,Near Mint (NM or M-),Very Good Plus (VG+),4.99,3.6,8.59,"{""Nothing's Gonna Stop Us Now"": 523408, 'Karma..."
4,6544ebd00d20ed70b40d2a9e,Jimmy Frey,Rosas A Sandra / Yo Soy De Carne Y Hueso,"Exit Records (4), EKIPO",Very Good Plus (VG+),Very Good (VG),0.99,3.6,4.59,{}


In [113]:
df = df.drop('_id', axis = 1)

In [114]:
df.to_csv('actualizado.csv', index=False)

# Traslado de datos a MongoDB

In [115]:
df = pd.read_csv('actualizado.csv')

In [116]:
df.head()

,author,album,label,item_condition,sleeve_condition,price,ship_price,final_price,song_rank
0,Kid Rock,The History Of Rock,"Lava, Lava, Atlantic, Top Dog Records",Near Mint (NM or M-),Near Mint (NM or M-),5.99,3.6,9.59,"{'Honeycomb': 13168, 'Honeycomb (1957)': 425, ..."
1,Kid Rock (2),Ic,"Poplandia, Young Blood",Near Mint (NM or M-),Very Good Plus (VG+),34.99,3.6,38.59,"{'I Only Have Eyes for You': 100000, 'Chrome H..."
2,Olga Ramos,Antologia Del Cuplé,Movieplay,Very Good Plus (VG+),Very Good Plus (VG+),2.99,3.6,6.59,"{'Colón 34': 19610, 'Ven y ven (Versión picare..."
3,Graham Parker,Protection,Vertigo,Near Mint (NM or M-),Very Good Plus (VG+),4.99,3.6,8.59,"{""Nothing's Gonna Stop Us Now"": 523408, 'Karma..."
4,Jimmy Frey,Rosas A Sandra / Yo Soy De Carne Y Hueso,"Exit Records (4), EKIPO",Very Good Plus (VG+),Very Good (VG),0.99,3.6,4.59,{}


In [117]:
str_conn = 'mongodb://localhost:27017'   # str_con por defecto

cursor = MongoClient(str_conn)

cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [131]:
db = cursor["sonic_harbor"]
collection = db.inventario

In [125]:
df['_id'] = range(1, len(df) + 1)

df = df[['_id', 'author', 'album', 'label', 'item_condition','sleeve_condition', 'price', 'ship_price', 'final_price', 'song_rank' ]]

In [129]:
df.head()

,_id,author,album,label,item_condition,sleeve_condition,price,ship_price,final_price,song_rank
0,1,Kid Rock,The History Of Rock,"Lava, Lava, Atlantic, Top Dog Records",Near Mint (NM or M-),Near Mint (NM or M-),5.99,3.6,9.59,"{'Honeycomb': 13168, 'Honeycomb (1957)': 425, ..."
1,2,Kid Rock (2),Ic,"Poplandia, Young Blood",Near Mint (NM or M-),Very Good Plus (VG+),34.99,3.6,38.59,"{'I Only Have Eyes for You': 100000, 'Chrome H..."
2,3,Olga Ramos,Antologia Del Cuplé,Movieplay,Very Good Plus (VG+),Very Good Plus (VG+),2.99,3.6,6.59,"{'Colón 34': 19610, 'Ven y ven (Versión picare..."
3,4,Graham Parker,Protection,Vertigo,Near Mint (NM or M-),Very Good Plus (VG+),4.99,3.6,8.59,"{""Nothing's Gonna Stop Us Now"": 523408, 'Karma..."
4,5,Jimmy Frey,Rosas A Sandra / Yo Soy De Carne Y Hueso,"Exit Records (4), EKIPO",Very Good Plus (VG+),Very Good (VG),0.99,3.6,4.59,{}


In [132]:
for nuevo in df.to_dict(orient="records"):
    collection.update_one({"_id": nuevo["_id"]}, {"$set": nuevo}, upsert=False)